In [1]:
import scglue.models.glue as glue
import scglue.models.sc as sc
from typing import List, Mapping, Optional, Tuple, Union
import torch
import torch.distributions as D
import torch.nn.functional as F
from scglue.num import EPS
import scglue.models.data as d
import scglue.models.scglue as sc
import anndata as ad
import networkx as nx
import scglue



# generate data 
rna = ad.read_h5ad("rna-pp2.h5ad")

atac = ad.read_h5ad("atac-pp2.h5ad")
guidance = nx.read_graphml("guidance2.graphml.gz")

scglue.models.configure_dataset(
    rna, "NB", use_highly_variable=True,
    use_layer="counts", use_rep="X_pca"
)

scglue.models.configure_dataset(
    atac, "NB", use_highly_variable=True,
    use_rep="X_lsi"
)
adatas={"rna": rna, "atac": atac}


In [2]:
import pandas as pd 
vertices=pd.Index(sorted(guidance.nodes))

In [3]:
pretrain_init_kws={}
pretrain_init_kws.update({"shared_batches": False})


pretrain = scglue.models.scglue.SCGLUEModel(adatas,sorted(guidance.nodes),**pretrain_init_kws)
# print(pretrain.net)
print(pretrain.net.keys)

[INFO] autodevice: Using CPU as computation device.
['rna', 'atac']


In [4]:
modalities=pretrain.modalities

In [5]:
#pretrain.fit(adatas, graph, **pretrain_fit_kws)
import os
import numpy as np
pretrain_fit_kws={'directory': 'glue'}
pretrain_fit_kws.update({"align_burnin": np.inf, "safe_burnin": False})

print(pretrain_fit_kws)

if "directory" in pretrain_fit_kws:
    pretrain_fit_kws["directory"] = os.path.join(pretrain_fit_kws["directory"], "pretrain")
    
pretrain_fit_kws

{'directory': 'glue', 'align_burnin': inf, 'safe_burnin': False}


{'directory': 'glue/pretrain', 'align_burnin': inf, 'safe_burnin': False}

In [6]:
#define compile_kws as empty 
compile_kws={}
pretrain.compile(**compile_kws)

In [7]:
#define dataloader and Anndata
from scglue.models.data import AnnDataset, ArrayDataset, DataLoader, GraphDataset

anndata = AnnDataset(
            [adatas[key] for key in pretrain.net.keys],
            [modalities[key] for key in pretrain.net.keys],
            mode="train"
        )

data_loader = DataLoader( #three elements: x, xrep, _
            anndata, batch_size=128,
            shuffle=False, drop_last=False
        )
data_config2=anndata._data_configs


In [8]:

#compute_losses 
lam_data=1
lam_kl=1.0
lam_graph=0.02
lam_align=0.05
lam_sup=0.02
modality_weight={'rna': 1.0, 'atac': 1.0}
normalize_u=False
lr=0.002
optim=str("RMSprop")
#train_step 
from scglue.models.scglue import SCGLUETrainer
import torch.optim as optim 

'''
net: SCGLUE, lam_data: float = None, lam_kl: float = None,
            lam_graph: float = None, lam_align: float = None,
            lam_sup: float = None, normalize_u: bool = None,
            modality_weight: Mapping[str, float] = None,
            optim: str = None, lr: float = None,'''

trainer=SCGLUETrainer(pretrain.net,lam_data,lam_kl,lam_graph,lam_align,lam_sup,normalize_u,modality_weight,"RMSprop",lr)


In [9]:
xuid, (x, xrep, xbch, xlbl, xdwt)=anndata._extract_data_train(data_config2)
extracted_data=anndata._extract_data_train(data_config2)

In [10]:
K=len(pretrain.net.keys)
#get x,xrep,xbch,xlbl,xdwt 
for data in data_loader:
    print(len(data))
#     print(data[2:4])
    x=data[0:2]
    print(x[1].shape)
    xrep=data[2:4]
    xbch=data[4:6]
    xlbl=data[6:8]
    xdwt=data[8:10]
    print(data[11:])

#     data2=trainer.format_data(data)
    x, xrep, xbch, xlbl, xdwt, (eidx, ewt, esgn) = \
            data[0:K], data[K:2*K], data[2*K:3*K], data[3*K:4*K], data[4*K:5*K], \
            data[5*K+1:]
    
    
    break

11
torch.Size([128, 12434])
()


ValueError: not enough values to unpack (expected 3, got 0)

In [ ]:
# anndata

In [ ]:
# from scglue.models.data import GraphDataset 
# gdata=GraphDataset(guidance,vertices)

In [ ]:

# graph = GraphDataset(
#             guidance, vertices, neg_samples=10,
#             weighted_sampling=True, deemphasize_loops=True
#         )


In [11]:
neg_samples=10
graph2 = GraphDataset(
            guidance, vertices, neg_samples=neg_samples,
            weighted_sampling=True, deemphasize_loops=True
        )
# gluetrainer.fit(data=anndata,graph2)


In [ ]:
# for g in graph:
#     print(len(g))
#     break

In [19]:
enorm.shape
# graph2.ewt

torch.Size([590267])

In [ ]:
adatas

In [23]:

trainer.fit(anndata,graph2,0.1,128,13106,float('inf'),True,2,28,14,1,0)
#val_split=0.1, data_batch_size 128, graph_batch_size 13106, align_burnin:inf, max_epochs:2

[INFO] SCGLUETrainer: Using training directory: "/var/folders/12/d54y_pk95zdbftjg_qkcj_c80000gq/T/GLUETMPpmupm578"
[INFO] EarlyStopping: No usable checkpoint found. Skipping checkpoint restoration.


In [48]:
from scglue.num import normalize_edges 
enorm = torch.as_tensor(
            normalize_edges(graph2.eidx, graph2.ewt),
        )
esgn=torch.as_tensor(graph2.esgn)
eidx=torch.as_tensor(graph2.eidx)
ewt=torch.as_tensor(graph2.ewt)
print(eidx)
trainer.eidx=eidx 
trainer.enorm=enorm
trainer.esgn=esgn
print(esgn[1])
print(esgn)
print(len(eidx[1]))
esgn

tensor([[ 28191,  28191,  28191,  ..., 165704, 179704,  66235],
        [142910, 142912, 142914,  ..., 165704, 179704,  66235]])
tensor(1.)
tensor([1., 1., 1.,  ..., 1., 1., 1.])
590267


tensor([1., 1., 1.,  ..., 1., 1., 1.])

In [41]:
from itertools import chain

for gdata in graph2:
#     print(len(gdata))
    eidx=gdata[0:1]
    eidx=list(chain.from_iterable(eidx))
    ewt=gdata[1:2][0]
#     ewt=list(chain.from_iterable(ewt))
    esgn=gdata[2:3][0]
#     esgn=list(chain.from_iterable(esgn))
    print(eidx)
    print(len(ewt),ewt)
    break
print(esgn)# one list not two 
print(ewt)# one list not two


[tensor([249617,  90600,  48241,  ..., 108604,  25272, 178207]), tensor([ 28299, 134766,  24836,  ..., 186945,  24267,   4718])]
3276 tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([1., 1., 1.,  ..., 1., 1., 1.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])


In [24]:
print(len(eidx[0]))
for i in ewt:
    print(len(i))

3276
3276


In [101]:
print(eidx[1])
eidx[0]


tensor([142910, 142912, 142914,  ..., 165704, 179704,  66235])


tensor([ 28191,  28191,  28191,  ..., 165704, 179704,  66235])

In [108]:
from itertools import chain
# x, xrep, xbch, xlbl, xdwt, (eidx, ewt, esgn)
data_list=[]
data_list.append(x[0])#1
data_list.append(x[1])#1
data_list.append(xrep[0])#2
data_list.append(xrep[1])#2
data_list.append(xbch[0])#3
data_list.append(xbch[1])#3
data_list.append(xlbl[0])#4
data_list.append(xlbl[1])#4
data_list.append(xdwt[0])#5
data_list.append(xdwt[1])#5
# data_list.append(pmsk)#6
data_list.append(eidx)#7 have to append eidx two times in order to include both eidx[0] and eidx[1]
data_list.append(eidx)#7
data_list.append(ewt)#8

data_list.append(esgn)#9
# data_list.append(esgn[1])#9

# data_list=list(chain.from_iterable(data_list))
print(len(data_list))

14


In [106]:
x

[tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]])]

In [103]:
esgn[0]
ewt[0]

tensor(1.)

In [98]:
data_list[11:]

[tensor([142910, 142912, 142914,  ..., 165704, 179704,  66235]),
 tensor([1., 1., 1.,  ..., 1., 1., 1.]),
 tensor([1., 1., 1.,  ..., 1., 1., 1.])]

In [90]:
sidx, tidx = eidx
eidx.shape

torch.Size([2, 590267])

In [74]:
# K=2
# x, xrep, xbch, xlbl, xdwt, (eidx, ewt, esgn) = \
#    data_list[0:K], data_list[K:2*K], data_list[2*K:3*K], data_list[3*K:4*K], data_list[4*K:5*K], \
#      data_list[5*K+1:]

ValueError: not enough values to unpack (expected 6, got 5)

In [109]:

DataTensor=trainer.format_data(data_list)
DataTensor

({'rna': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]),
  'atac': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]])},
 {'rna': tensor([[ 3.4848e+00,  1.1419e+01,  9.0474e+00,  ..., -5.3635e+00,
            1.6005e-01, -1.2033e+00],
          [ 1.9448e-01, -4.5760e-01,  1.2121e+00,  ...,  7.5727e-02,
           -1.0081e-02, -4.6408e-02],
          [-1.4876e+00, -2.0016e+00,  1.0567e+00,  ..., -2.0454e-01,
            2.3666e-01, -3.0249e-01],
          ...,
          [-1.8596e+00, -1.6021e+00,  7.4491e-02,  ...,  1.5663e-01,
            7.0951e-01,  2.2479e-0

In [ ]:
# print(trainer.eidx)
# trainer.eidx=eidx
# trainer.ewt=ewt
# trainer.esgn=esgn
# print(trainer.enorm)

In [ ]:
import os
print(os.getcwd())
os.chdir('/Users/meiqiliu/PycharmProjects/GLUE')


In [113]:
x2, xrep2, xbch2, xlbl2, xdwt2, xflag2, eidx2, ewt2, esgn2 = DataTensor
eidx2[0]
eidx2[1]

eidx2

tensor([[ 28191,  28191,  28191,  ..., 165704, 179704,  66235],
        [142910, 142912, 142914,  ..., 165704, 179704,  66235]])

In [115]:
a,b=eidx2
a

tensor([ 28191,  28191,  28191,  ..., 165704, 179704,  66235])

In [110]:
trainer.compute_losses(DataTensor,2) #function with loss functions 

{'dsc_loss': tensor(0.6945, grad_fn=<DivBackward0>),
 'vae_loss': tensor(0.2606, grad_fn=<AddBackward0>),
 'gen_loss': tensor(0.2259, grad_fn=<SubBackward0>),
 'g_nll': tensor(0.2377, grad_fn=<DivBackward0>),
 'g_kl': tensor(0.0001, grad_fn=<DivBackward0>),
 'g_elbo': tensor(0.2378, grad_fn=<AddBackward0>),
 'x_rna_nll': tensor(0.2004, grad_fn=<NegBackward0>),
 'x_rna_kl': tensor(0.0121, grad_fn=<DivBackward0>),
 'x_rna_elbo': tensor(0.2125, grad_fn=<AddBackward0>),
 'x_atac_nll': tensor(0.0379, grad_fn=<NegBackward0>),
 'x_atac_kl': tensor(0.0007, grad_fn=<DivBackward0>),
 'x_atac_elbo': tensor(0.0386, grad_fn=<AddBackward0>)}